### Install and import packages

In [3]:
!pip install opencv-python
!pip install cv2

   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   - -------------------------------------- 1.0/39.5 MB 6.3 MB/s eta 0:00:07
   --- ------------------------------------ 3.1/39.5 MB 8.4 MB/s eta 0:00:05
   ----- ---------------------------------- 5.5/39.5 MB 9.1 MB/s eta 0:00:04
   ------- -------------------------------- 7.9/39.5 MB 9.7 MB/s eta 0:00:04
   ---------- ----------------------------- 10.5/39.5 MB 10.4 MB/s eta 0:00:03
   ------------- -------------------------- 13.4/39.5 MB 10.9 MB/s eta 0:00:03
   ---------------- ----------------------- 16.3/39.5 MB 11.4 MB/s eta 0:00:03
   ------------------- -------------------- 19.4/39.5 MB 11.8 MB/s eta 0:00:02
   ---------------------- ----------------- 22.5/39.5 MB 12.2 MB/s eta 0:00:02
   -------------------------- ------------- 25.7/39.5 MB 12.4 MB/s eta 0:00:02
   ----------------------------- ---------- 28.8/39.5 MB 12.6 MB/s eta 0:00:01
   -------------------------------- ------- 32.2/39.5 MB 12.9 MB/s et

ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2


In [4]:
# import cudf
import numpy as np
import pandas as pd
from PIL import Image
import cv2
import os
import tqdm
from skimage.feature import hog
# from cuml.model_selection import train_test_split
# from cuml.metrics import accuracy_score
# from cuml.svm import SVC
import torch
from torchvision import models, transforms, datasets 
from torch.utils.data import DataLoader
import warnings
warnings.filterwarnings("ignore")

In [2]:
import cuml
print(cuml.__version__)

25.06.00


### Loading image summary

Do note that the pathnames have been changed to assume that you are in the `CS610_AML_Group_Project` directory

In [5]:
image_csv = pd.read_csv('../CS610_AML_Group_Project/dataset_stats.csv')
image_csv.head()

,class,image_count,avg_width,avg_height,min_width,min_height,max_width,max_height,formats,corrupt_files
0,adidas_forum_high,150,143,124,78,81,162,140,jpeg,0
1,adidas_ultraboost,150,142,128,93,49,162,140,jpeg,0
2,new_balance_550,150,134,129,79,40,162,140,jpeg,0
3,new_balance_574,150,131,133,78,68,162,140,jpeg,0
4,converse_one_star,150,138,130,82,67,162,140,jpeg,0


In [4]:
# check images format
image_csv['formats'].unique()

array(['jpeg', 'jpeg, png'], dtype=object)

In [8]:
def count_images(datasource_path):
    image_counts = {}
    image_extensions = {'.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff', '.webp'}

    if not os.path.isdir(datasource_path):
        print(f"Error: Path '{datasource_path}' is not a directory.")
        return image_counts

    for subfolder_name in os.listdir(datasource_path):
        subfolder_path = os.path.join(datasource_path, subfolder_name)

        if os.path.isdir(subfolder_path):
            count = 0
            for file_name in os.listdir(subfolder_path):
                file_path = os.path.join(subfolder_path, file_name)
                if os.path.isfile(file_path):
                    _, ext = os.path.splitext(file_name)
                    if ext.lower() in image_extensions:
                        count += 1
            image_counts[subfolder_name] = count
    return image_counts

# --- 请根据你的实际情况修改以下路径 ---
# 假设你的datasource文件夹结构如图片所示：
# 你的项目根目录 -> Complementary Dataset -> datasource
# 并且你的Python脚本位于项目根目录或者可以通过相对路径找到

# 获取当前脚本的目录
image_dir = 'datasource'
print(f"Scanning: {image_dir}")
counts = count_images(image_dir)

if counts:
    for folder, count in counts.items():
        print(f"{folder}: {count} images")
else:
    print("No images found or path is incorrect/empty.")
# else:

Scanning: datasource
adidas_forum_high: 150 images
adidas_forum_low: 115 images
adidas_gazelle: 149 images
adidas_nmd_r1: 115 images
adidas_samba: 115 images
adidas_stan_smith: 147 images
adidas_superstar: 115 images
adidas_ultraboost: 150 images
asics_gel-lyte_iii: 91 images
converse_chuck_70_high: 115 images
converse_chuck_70_low: 148 images
converse_chuck_taylor_all-star_high: 114 images
converse_chuck_taylor_all-star_low: 114 images
converse_one_star: 150 images
new_balance_327: 108 images
new_balance_550: 150 images
new_balance_574: 150 images
new_balance_990: 113 images
new_balance_992: 150 images
nike_air_force_1_high: 115 images
nike_air_force_1_low: 147 images
nike_air_force_1_mid: 148 images
nike_air_jordan_11: 113 images
nike_air_jordan_1_high: 115 images
nike_air_jordan_1_low: 115 images
nike_air_jordan_3: 100 images
nike_air_jordan_4: 150 images
nike_air_max_1: 106 images
nike_air_max_270: 149 images
nike_air_max_90: 150 images
nike_air_max_95: 115 images
nike_air_max_97: 

### Image Processing

#### Resizing

In [5]:
# resize images
def resize_image_in_folder(input_dir, output_dir, size=(224, 224), desc='resizing images'):
    if not os.path.exists(input_dir):
        print(f"Input directory {input_dir} does not exist. Please check the path.")
        return

    os.makedirs(output_dir, exist_ok=True)
    supported_formats = ('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff', '.webp')
    for filename in os.listdir(input_dir):
        if filename.lower().endswith(supported_formats):
            img_input_path = os.path.join(input_dir, filename)
            img_output_path = os.path.join(output_dir, filename)
            try:
                img = cv2.imread(img_input_path, cv2.IMREAD_UNCHANGED)
                
                if img is None:
                    print(f"Error loading {img_input_path}")
                    continue
                resized_img = cv2.resize(img, size, interpolation=cv2.INTER_LANCZOS4)

                if img_output_path.lower().endswith(('.jpg', '.jpeg')) and resized_img.shape[-1] == 4:
                    resized_img = cv2.cvtColor(resized_img, cv2.COLOR_BGRA2BGR)
                cv2.imwrite(img_output_path, resized_img)
            except Exception as e:
                print(f"Error processing {img_input_path}: {e}")

In [6]:
# process all folders
def batch_resize_images(base_input_dir, base_output_dir, size=(224, 224)):
    if not os.path.exists(base_input_dir):
        print(f"Base directory {base_input_dir} does not exist. Please check the path.")
        return
    
    os.makedirs(base_output_dir, exist_ok=True) # if output directory does not exist, create it.
    
    for folder in tqdm.tqdm(os.listdir(base_input_dir)):
        current_input_subfolder = os.path.join(base_input_dir, folder)
        current_output_subfolder = os.path.join(base_output_dir, folder)
        
        if os.path.isdir(current_input_subfolder):
            resize_image_in_folder(current_input_subfolder, current_output_subfolder, size=size)
        else:
            print(f"Skipping {current_input_subfolder} as it is not a directory.")
    
    print("Batch resizing completed.")

In [ ]:
base_input_dir = '../CS610_AML_Group_Project/datasource'
base_output_dir = '../CS610_AML_Group_Project/resized_images'

In [8]:
batch_resize_images(base_input_dir, base_output_dir, size=(224, 224))

100%|██████████| 50/50 [00:05<00:00,  8.70it/s]

Batch resizing completed.


#### Gray Scaling

In [42]:
def grayscale_image_in_folder(input_dir, output_dir):
    if not os.path.exists(input_dir):
        print(f"Input directory {input_dir} does not exist. Please check the path.")
        return

    os.makedirs(output_dir, exist_ok=True)
    supported_formats = ('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff', '.webp')
    for filename in os.listdir(input_dir):
        if filename.lower().endswith(supported_formats):
            img_input_path = os.path.join(input_dir, filename)
            img_output_path = os.path.join(output_dir, filename)
            try:
                img = cv2.imread(img_input_path)
                if img is None:
                    print(f"Error loading {img_input_path}")
                    continue
                # Convert to grayscale
                gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                cv2.imwrite(img_output_path, gray_img)
            except Exception as e:
                print(f"Error processing {img_input_path}: {e}")

In [43]:
def batch_grayscale_images(base_input_dir, base_output_dir):
    if not os.path.exists(base_input_dir):
        print(f"Base directory {base_input_dir} does not exist. Please check the path.")
        return

    os.makedirs(base_output_dir, exist_ok=True)

    for folder in tqdm.tqdm(os.listdir(base_input_dir)):
        current_input_subfolder = os.path.join(base_input_dir, folder)
        current_output_subfolder = os.path.join(base_output_dir, folder)

        if os.path.isdir(current_input_subfolder):
            grayscale_image_in_folder(current_input_subfolder, current_output_subfolder)
        else:
            print(f"Skipping {current_input_subfolder} as it is not a directory.")

    print("Batch grayscale completed.")

In [ ]:
base_input_dir = '../CS610_AML_Group_Project/resized_images'
base_output_dir = '../CS610_AML_Group_Project/grayscale_images'
batch_grayscale_images(base_input_dir, base_output_dir)

100%|██████████| 50/50 [00:01<00:00, 28.81it/s]

Batch grayscale completed.


#### Flatten

In [5]:
def flatten_images_recursive(input_dir):
    features = []
    filenames = []
    supported_formats = ('.png', '.jpg', '.jpeg', '.bmp', '.tiff', '.webp')
    for root, dirs, files in os.walk(input_dir):
        for filename in files:
            if filename.lower().endswith(supported_formats):
                img_path = os.path.join(root, filename)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img is None:
                    continue
                img_flat = img.flatten()
                features.append(img_flat)
                # Save relative path for label or traceability
                rel_path = os.path.relpath(img_path, input_dir)
                filenames.append(rel_path)
    flattened = np.array(features)
    return flattened, filenames

# Example usage:
input_dir = '../CS610_AML_Group_Project/grayscale_images'
flattened, filenames = flatten_images_recursive(input_dir)
print(flattened.shape)  # (num_images, 224*224)

(5953, 50176)


## Feature extraction

#### Method 1 - By HOG

In [6]:
from skimage.feature import hog

def extract_hog_features_recursive(input_dir, pixels_per_cell=(16, 16), cells_per_block=(2, 2)):
    features = []
    filenames = []
    supported_formats = ('.png', '.jpg', '.jpeg', '.bmp', '.tiff', '.webp')
    for root, dirs, files in os.walk(input_dir):
        for filename in files:
            if filename.lower().endswith(supported_formats):
                img_path = os.path.join(root, filename)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img is None:
                    continue
                # Extract HOG features
                hog_feature = hog(img, pixels_per_cell=pixels_per_cell, cells_per_block=cells_per_block, feature_vector=True)
                features.append(hog_feature)
                rel_path = os.path.relpath(img_path, input_dir)
                filenames.append(rel_path)
    hogged = np.array(features)
    return hogged, filenames

# Example usage:
input_dir = '../CS610_AML_Group_Project/grayscale_images'
hogged, filenames = extract_hog_features_recursive(input_dir)
print(hogged.shape)  # (num_images, hog_feature_dim)

(5953, 6084)


### Method 2 - Using pretrained CNN

ResNet50 will be used as the feature extractor due to its pre-trained weights, derived from large datasets like ImageNet, and is a popular choice to use for computer vision applications such as image classification.
Reference:
1) https://medium.com/@meetkalathiya1301/feature-extraction-using-pre-trained-models-for-image-classification-16e6ff43f268
2) https://stackoverflow.com/questions/62117707/extract-features-from-pretrained-resnet50-in-pytorch

In [7]:
#Process image data for feature extraction using CNN
base_output_dir = '../CS610_AML_Group_Project/resized_images'
img_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])]) #mean and std based on ImageNet - normalise image data closer to normal distribution
img_dataset = datasets.ImageFolder(base_output_dir, transform=img_transform)
data_loader = DataLoader(img_dataset, batch_size=32, num_workers=4)  

input_dir = '../CS610_AML_Group_Project/grayscale_images'

In [8]:
#define function for CNN feature extraction
def cnn_feature_extract(cnn_feature_extractor, data_loader):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    #prepare cnn model to use for feature extraction
    cnn_feature_extractor.eval()
    cnn_feature_extractor.fc = torch.nn.Identity() #replace fully connected layer of pretrained cnn with Identity layer
    for para in cnn_feature_extractor.parameters():
        para.requires_grad = False #freeze weights
    #feature extraction
    features_list, labels_list = [], []
    cnn_feature_extractor.to(device)
    with torch.no_grad():
        for images, labels in data_loader:
            images = images.to(device)
            feature = cnn_feature_extractor(images)
            feature = feature.view(feature.size(0),-1) #flatten into (n_samples, n_features) for non-CNN models
            #convert tensors into numpy for fitting into non-CNN models and add into lists
            features_list.append(feature.cpu().numpy())
            labels_list.append(labels.numpy())

    return cnn_feature_extractor, np.vstack(features_list), np.hstack(labels_list)

In [9]:
#initialise and extract features using CNN feature extractor
weights = models.ResNet50_Weights.IMAGENET1K_V2
resnet50_extractor = models.resnet50(weights=weights)
resnet50_extractor, X, y = cnn_feature_extract(resnet50_extractor, data_loader) #X = features, y =labels
#no need labelling as the numpy array is generated from the data_loader

In [11]:
#CNN training and test split
x_train, x_test, y_encoded_train, y_encoded_test = train_test_split(X,y, test_size=0.2, random_state=42, stratify=y)
x_train_cpu = pd.DataFrame(x_train, dtype=np.float32)
y_train_cpu = pd.Series(y_encoded_train, dtype=np.int32)
x_test_cpu = pd.DataFrame(x_test, dtype=np.float32)
y_test_cpu = pd.Series(y_encoded_test, dtype=np.int32)
#same as original flow
print("Number of Samples:", len(y_train_cpu))
print("Number of Labels:", len(np.unique(y_train_cpu)))
counts = y_train_cpu.value_counts()
print("Label Distribution:")
print(counts)

Number of Samples: 4763
Number of Labels: 50
Label Distribution:
0     120
7     120
15    120
16    120
13    120
36    120
37    120
29    120
26    120
18    120
21    119
33    119
28    119
10    119
2     119
48    119
45    119
43    119
39    118
41    118
5     118
42    118
20    118
49    116
34     86
47     86
14     86
27     85
25     80
40     78
31     78
35     78
3      76
17     76
22     74
44     74
1      74
32     74
6      74
12     74
8      73
46     70
19     70
30     69
38     64
24     63
11     62
23     62
9      60
4      59
Name: count, dtype: int64


In [4]:
#Stacking - to use for 
from cuml import LogisticRegression
from cuml.ensemble import RandomForestClassifier
from cuml.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier
import xgboost as xgb

estimators = [('rcf_model',RandomForestClassifier(random_state=42)),("xgboost",xgb.XGBClassifier(random_state=42)),("knn", KNeighborsClassifier())]
stacking_cf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(), cv=5, passthrough=False, verbose=1)
#split dataset into training and test dataset and fit into model via cross_val_score (cross validation approach)
stacking_cf.fit(x_train,y_encoded_train)
print("Accuracy Score:",stacking_cf.score)

AttributeError: partially initialized module 'cuml' has no attribute 'internals' (most likely due to a circular import)

#### Labelling

In [46]:
y = [f.split(os.sep)[0] for f in filenames]

#### Data Splitting

In [48]:
x = hogged.astype(np.float32)
y = np.array(y)
y_encoded, uniques = pd.factorize(y)

In [18]:
x_train, x_test, y_encoded_train, y_encoded_test = train_test_split(x, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

In [58]:
x_train_gpu = cudf.DataFrame(x_train, dtype=np.float32)
y_train_gpu = cudf.Series(y_encoded_train, dtype=np.int32)
x_test_gpu = cudf.DataFrame(x_test, dtype=np.float32)
y_test_gpu = cudf.Series(y_encoded_test, dtype=np.int32)
x_train_cpu = pd.DataFrame(x_train, dtype=np.float32)
y_train_cpu = pd.Series(y_encoded_train, dtype=np.int32)
x_test_cpu = pd.DataFrame(x_test, dtype=np.float32)
y_test_cpu = pd.Series(y_encoded_test, dtype=np.int32)

In [21]:
print("Number of Samples:", len(y_train_cpu))
print("Number of Labels:", len(np.unique(y_train_cpu)))


counts = y_train_cpu.value_counts()
print("Label Distribution:")
print(counts)

Number of Samples: 4763
Number of Labels: 50
Label Distribution:
0     120
2     120
14    120
6     120
11    120
17    120
10    120
13    120
37    120
45    120
48    119
29    119
49    119
41    119
26    119
21    119
9     119
8     119
30    118
28    118
32    118
19    118
3     118
43    116
5      86
7      86
24     86
34     85
47     80
31     78
20     78
25     78
40     76
38     76
46     74
18     74
22     74
12     74
16     74
27     74
1      73
36     70
39     70
15     69
35     64
4      63
42     62
33     62
23     60
44     59
Name: count, dtype: int64


#### SVM

In [20]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_cpu = scaler.fit_transform(x_train_cpu)

In [21]:
%load_ext cuml.accel  
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform, uniform


param_dist = {
    'C': loguniform(1e-3, 1e3),      
    'kernel': ['linear']            
}


svm = SVC(cache_size=4096)  


random_search = RandomizedSearchCV(
    estimator=svm,
    param_distributions=param_dist,
    n_iter=1,  
    scoring='accuracy',
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=1    
)

random_search.fit(x_train_cpu, y_train_cpu) 
print("Best params:", random_search.best_params_)
print("Best score:", random_search.best_score_)


The cuml.accel extension is already loaded. To reload it, use:
  %reload_ext cuml.accel
Fitting 3 folds for each of 1 candidates, totalling 3 fits
CUBLAS call='cublasCreate(&cublas_res)' at file=/tmp/conda-bld-output/bld/rattler-build_libcuml/host_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehol/include/raft/core/resource/cublas_handle.hpp line=31 failed with CUBLAS_STATUS_ALLOC_FAILED
CUBLAS call='cublasSetStream(cublas_res, stream)' at file=/tmp/conda-bld-output/bld/rattler-build_libcuml/host_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehol/include/raft/core/resource/cublas_handle.hpp line=32 failed with CUBLAS_STATUS_NOT_INITIALIZED
[CV] END ................C=0.1767016940294795, k

ValueError: 
All the 3 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/home/iamivanwu/miniconda3/envs/rapids-25.06/lib/python3.13/site-packages/sklearn/model_selection/_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/iamivanwu/miniconda3/envs/rapids-25.06/lib/python3.13/site-packages/cuml/internals/api_decorators.py", line 216, in wrapper
    ret = func(*args, **kwargs)
  File "cuml/internals/base.pyx", line 98, in cuml.internals.base.deprecate_non_keyword_only.decorator.inner
  File "cuml/svm/svc.pyx", line 600, in cuml.svm.svc.SVC.fit
  File "/home/iamivanwu/miniconda3/envs/rapids-25.06/lib/p

#### Decision Tree

In [ ]:
%load_ext cuml.accel
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
from sklearn.model_selection import StratifiedKFold


param_dist = {
    'max_depth': randint(5, 25),  
    'min_samples_split': randint(10, 50),
    'criterion': ['gini', 'entropy'],
    'min_impurity_decrease': uniform(0, 0.05)
}

dt = DecisionTreeClassifier(random_state=42)
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
random_search = RandomizedSearchCV(
    estimator=dt,
    param_distributions=param_dist,
    n_iter=10,
    scoring='accuracy',
    cv=cv,
    random_state=42,
    n_jobs=-1
)

print("feature matrix", x_train_cpu.shape)
print("label distribution:", np.unique(y_train_cpu, return_counts=True))

random_search.fit(x_train_cpu, y_train_cpu)



cuML: Accelerator installed.
feature matrix (4763, 6084)
label distribution: (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
      dtype=int32), array([120,  73, 120, 118,  63,  86, 120,  86, 119, 119, 120, 120,  74,
       120, 120,  69,  74, 120,  74, 118,  78, 119,  74,  60,  86,  78,
       119,  74, 118, 119, 118,  78, 118,  62,  85,  64,  70, 120,  76,
        70,  76, 119,  62, 116,  59, 120,  74,  80, 119, 119]))


/home/iamivanwu/miniconda3/envs/rapids-25.06/lib/python3.13/site-packages/sklearn/utils/_response.py:203: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  target_type = type_of_target(classes)
/home/iamivanwu/miniconda3/envs/rapids-25.06/lib/python3.13/site-packages/sklearn/utils/_response.py:203: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  target_type = type_of_target(classes)
/home/iamivanwu/miniconda3/envs/rapids-25.06/lib/python3.13/site-packages/sklearn/utils/_response.py:203: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  target_type = type_of_target(classes)
/home/iamivanwu/miniconda3/envs/rapids-25.06/lib/python3.13/site-packages/sklearn/utils/_response.py:203: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  target_type = type_of_target(classes)
/home/iamivanwu/miniconda3/envs/rapids-25.06/lib/python3.13/site

,estimator,DecisionTreeC...ndom_state=42)
,param_distributions,"{'criterion': ['gini', 'entropy'], 'max_depth': <scipy.stats....x7f97360738c0>, 'min_impurity_decrease': <scipy.stats....x7f9736073a10>, 'min_samples_split': <scipy.stats....x7f973607ae90>}"
,n_iter,10
,scoring,'accuracy'
,n_jobs,-1
,refit,True
,cv,StratifiedKFo... shuffle=True)
,verbose,0
,pre_dispatch,'2*n_jobs'
,random_state,42
,error_score,nan


In [23]:
print("Best params:", random_search.best_params_)
print("Best score:", random_search.best_score_)

Best params: {'criterion': 'entropy', 'max_depth': 7, 'min_impurity_decrease': np.float64(0.0010292247147901223), 'min_samples_split': 11}
Best score: 0.08586915518986389


#### Random Forest

In [27]:
%load_ext cuml.accel
from cuml.ensemble import RandomForestClassifier
from cuml.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

param_dist = {
    'n_estimators': randint(50, 200),
    'max_depth': randint(5, 30),
    'max_features': uniform(0.5, 0.5)
}

rf = RandomForestClassifier()
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=10,
    scoring='accuracy',
    cv=3,
    verbose=2,
    random_state=42
)

random_search.fit(x_train_cpu, y_train_cpu)
print("Best params:", random_search.best_params_)
print("Best score:", random_search.best_score_)

cuML: Could not enable managed memory.
cuML: Installed accelerator for sklearn.
cuML: Successfully initialized accelerator.
Unused keyword parameter: ccp_alpha during cuML estimator initialization
Unused keyword parameter: monotonic_cst during cuML estimator initialization
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Unused keyword parameter: ccp_alpha during cuML estimator initialization
Unused keyword parameter: monotonic_cst during cuML estimator initialization
[CV] END max_depth=11, max_features=0.8982714934301164, n_estimators=64; total time=  28.1s
[CV] END max_depth=11, max_features=0.8982714934301164, n_estimators=64; total time=  26.4s
[CV] END max_depth=11, max_features=0.8982714934301164, n_estimators=64; total time=  26.1s
[CV] END max_depth=15, max_features=0.8898455001363846, n_estimators=70; total time=  45.6s
[CV] END max_depth=15, max_features=0.8898455001363846, n_estimators=70; total time=  43.7s
[CV] END max_depth=15, max_features=0.8898455001363846,

#### XGBoost

In [26]:
%load_ext cuml.accel
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

param_dist = {
    'n_estimators': randint(50, 200),
    'max_depth': randint(5, 30),
    'learning_rate': uniform(0.01, 0.3),  
    'subsample': uniform(0.6, 0.4),      
    'colsample_bytree': uniform(0.6, 0.4), 
    'gamma': uniform(0, 0.5)             
}


model = xgb.XGBClassifier(
    device="cuda",        
    tree_method="hist", 
    random_state=42
)

random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    n_iter=2,
    scoring='accuracy',
    cv=3,
    verbose=2,
    random_state=42,
    error_score='raise'
)

random_search.fit(x_train_cpu, y_train_cpu)
print("Best params:", random_search.best_params_)
print("Best score:", random_search.best_score_)


cuML: Accelerator installed.
Fitting 3 folds for each of 2 candidates, totalling 6 fits


/home/iamivanwu/miniconda3/envs/rapids-25.06/lib/python3.13/site-packages/sklearn/utils/_response.py:203: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  target_type = type_of_target(classes)
/home/iamivanwu/miniconda3/envs/rapids-25.06/lib/python3.13/site-packages/xgboost/core.py:705: UserWarning: [14:39:01] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1748362148159/work/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  return func(**kwargs)


[CV] END colsample_bytree=0.749816047538945, gamma=0.4753571532049581, learning_rate=0.22959818254342154, max_depth=25, n_estimators=152, subsample=0.7783331011414365; total time= 2.5min


/home/iamivanwu/miniconda3/envs/rapids-25.06/lib/python3.13/site-packages/sklearn/utils/_response.py:203: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  target_type = type_of_target(classes)


[CV] END colsample_bytree=0.749816047538945, gamma=0.4753571532049581, learning_rate=0.22959818254342154, max_depth=25, n_estimators=152, subsample=0.7783331011414365; total time= 2.5min


...

[CV] END colsample_bytree=0.6399899663272012, gamma=0.22962444598293358, learning_rate=0.11011258334170654, max_depth=12, n_estimators=180, subsample=0.608233797718321; total time= 4.1min


/home/iamivanwu/miniconda3/envs/rapids-25.06/lib/python3.13/site-packages/sklearn/utils/_response.py:203: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  target_type = type_of_target(classes)


[CV] END colsample_bytree=0.6399899663272012, gamma=0.22962444598293358, learning_rate=0.11011258334170654, max_depth=12, n_estimators=180, subsample=0.608233797718321; total time= 4.0min


/home/iamivanwu/miniconda3/envs/rapids-25.06/lib/python3.13/site-packages/sklearn/utils/_response.py:203: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  target_type = type_of_target(classes)


[CV] END colsample_bytree=0.6399899663272012, gamma=0.22962444598293358, learning_rate=0.11011258334170654, max_depth=12, n_estimators=180, subsample=0.608233797718321; total time= 4.1min
Best params: {'colsample_bytree': np.float64(0.6399899663272012), 'gamma': np.float64(0.22962444598293358), 'learning_rate': np.float64(0.11011258334170654), 'max_depth': 12, 'n_estimators': 180, 'subsample': np.float64(0.608233797718321)}
Best score: 0.2592917528385809
